In [2]:
import pandas as pd
from nltk.corpus import comparative_sentences  #keyword comparison 생성 모듈

data = pd.read_csv('pre_processingVer5.csv')
data = data.drop(['Unnamed: 0'], axis = 1)
data.head()

,asin,date,model,product,review,score,user
0,B07S1HTSGH,"June 30, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I am so glad I found this inexpensive camera o...,5.0,Arsenals
1,B07S1HTSGH,"June 30, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,The pictures came out very clear and very real...,5.0,Arsenals
2,B07S1HTSGH,"June 25, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,Now is a good time to travel.I'm a novice phot...,5.0,Vargas Laurence
3,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,I'm not sure if it was just the camera that I ...,3.0,Diane
4,B07S1HTSGH,"July 20, 2019",NaN,Digital Camera with WiFi 24.0 MP Vlogging Came...,When I transferred the pictures and videos to ...,3.0,Diane


In [3]:
#데이터 확인
len(data)

70976

In [13]:
#kw 사전 리스트 생성
### kw_list = comparative_sentences.keywords_readme()[5:]
### kw_str = '|'.join(kw_list)

#kw 사전 수작업으로 분류 후 다시 읽어오기
kw_list_excel = pd.read_excel('kw_list.xls')
#칼럼 중에 'to_use'만 가져오기
kw_list = list(kw_list['to_use'])
kw_str = '|'.join(kw_list)

kw_list

['beat',
 'inferior',
 'outstrip',
 'Choice',
 'choose',
 'prefer',
 'recommend',
 'outperform',
 'superior',
 'all',
 'favor',
 'defeat',
 'twice',
 'thrice',
 'Number one',
 'more',
 'like',
 'Versus',
 'first',
 'outdistance',
 'before',
 'double',
 'outsell',
 'nobody',
 'Vs',
 'last',
 'after',
 'thrice',
 'improve',
 'only',
 'outmatch',
 'ahead',
 'fraction',
 'outdo',
 'unmatched',
 'peerless',
 'differ',
 'outwit',
 'Compare',
 'top',
 'exceed',
 'lead',
 'win',
 'outstrip',
 'none',
 'unrivaled',
 'dominate',
 'second',
 'nonpareil',
 'advantage',
 'unlike',
 'least',
 'outclass',
 'outfox',
 'outdistance',
 'most',
 'up against',
 'less',
 'half',
 'behind',
 'Compete']

In [14]:
def make_sscs(data):
    from nltk import word_tokenize, pos_tag
    cs_list = []
    ss_list = []
    for i in range(len(data)):
        text = data.loc[i, 'review']
        product = data.loc[i, 'product']
        
        # nltk로 토크나이즈, 포스태깅
        tokens = word_tokenize(text)
        pos_tokens = pos_tag(tokens)
        
        # 비교구문이 들어있는 문장 찾기
        count = 0
        for tok, pos in pos_tokens:
            if pos in ['JJR', 'JJS', 'RBS', "RBR'"]:
                count +=1

        if count != 0:
            cs_list.append(i)
        elif count == 0:
            ss_list.append(i)
        
    
    df_ss = data.loc[ss_list]     
    
    df_cs = data.loc[cs_list]
    # product_list가 없어서 #처리함. 추후 보강해야 함
#     product_list = []
#     products = '|'.join(product_list)
#     df = df[df.review.str.contains(products)== True]
    df_cs = df_cs[df_cs.review.str.contains(kw_str)== True]
    
    df_cs['CS_SS'] = 'CS'
    df_ss['CS_SS'] = 'SS'

    df_t = df_cs.append(df_ss, ignore_index=True)
    return df_t

In [15]:
df_t = make_sscs(data)

In [16]:
len(df_t[df_t.CS_SS=='CS'])

7528

In [17]:
len(df_t[df_t.CS_SS=='SS'])

60520

In [11]:
df_t.to_excel('labelingSSCS.xlsx')